<a target="_blank" href="https://shiyanlou.com/louplus/ml"><img style="float: right;" src="https://doc.shiyanlou.com/courses/uid214893-20190709-1562652991830"></a>

## 构建 LeNet-5 Estimator

---

<i class="fa fa-exclamation-circle" aria-hidden="true"> 以下内容仅包含挑战需学员补充部分的参考答案，完整挑战请到原课程页面查看。</i>

---

In [ ]:
model = tf.keras.models.Sequential([
    # 卷积层，6 个 5x5 卷积核，步长为 1，relu 激活，第一层需指定 input_shape
    tf.keras.layers.Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1),
                           activation='relu', input_shape=(32, 32, 1)),
    # 平均池化，池化窗口默认为 2
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
    # 卷积层，16 个 5x5 卷积核，步为 1，relu 激活
    tf.keras.layers.Conv2D(filters=16, kernel_size=(
        5, 5), strides=(1, 1), activation='relu'),
    # 平均池化，池化窗口默认为 2
    tf.keras.layers.AveragePooling2D(pool_size=(2, 2), strides=2),
    # 需展平后才能与全连接层相连
    tf.keras.layers.Flatten(),
    # 全连接层，输出为 120，relu 激活
    tf.keras.layers.Dense(units=120, activation='relu'),
    # 全连接层，输出为 84，relu 激活
    tf.keras.layers.Dense(units=84, activation='relu'),
    # 全连接层，输出为 10，Softmax 激活
    tf.keras.layers.Dense(units=10, activation='softmax')
])
# 编译模型，Adam 优化器，多分类交叉熵损失函数，准确度评估
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['sparse_categorical_accuracy'])
# 查看网络结构
model.summary()

定义数据输入函数，根据官方指南说明，返回值务必是 [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)，同时请务必将 `tf.data.Dataset` 构建代码放置于函数中，否则后续训练时会报错。

In [ ]:
def train_input_fn():
    # 训练数据输入函数
    train_dataset = tf.data.Dataset.from_tensor_slices(
        (x_train.astype('float32'), y_train.astype('int64')))
    dataset = train_dataset.batch(32).repeat()
    return dataset


def test_input_fn():
    # 测试数据输入函数
    test_dataset = tf.data.Dataset.from_tensor_slices(
        (x_test.astype('float32'), y_test.astype('int64')))
    dataset = test_dataset.batch(32).repeat()
    return dataset

创建临时文件夹，并将 Keras 模型转换为 Estimator 模型。

In [ ]:
import tempfile

model_dir = tempfile.mkdtemp()
keras_estimator = tf.keras.estimator.model_to_estimator(
    keras_model=model, model_dir=model_dir)
keras_estimator

训练：

In [ ]:
keras_estimator.train(input_fn=train_input_fn, steps=500)
eval_result = keras_estimator.evaluate(input_fn=test_input_fn, steps=10)
print('Eval result: {}'.format(eval_result))

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>

Kaggle 完整运行过程：https://www.kaggle.com/huhuhang/lenet-5-estimator